# MODEL


In [1]:
import csv
import json
import pandas as pd

import random

from dotenv import load_dotenv
import os
import re
load_dotenv(override=True)

import time
from datetime import datetime 

from collections import defaultdict

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
 
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [2]:
# initialize variables
import pandas as pd
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# mysql credentials
PASSWORD = os.getenv("PASSWORD")
USER = os.getenv("USER")


load_dotenv(override=True)


True

In [3]:
# connect to db
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user=USER,
  password=PASSWORD,
  database="mpp21"
)

mycursor = mydb.cursor()

In [4]:
vector = CountVectorizer(stop_words='english') # count_vectorizer
NUMBER_TOPICS = 1
NUMBER_TOPICS2 = 10
MODEL_WORDS = 50 #50

In [5]:
def model(cnt):
    frequencies = vector.fit_transform(cnt)

    lda = LDA(n_components=NUMBER_TOPICS, n_jobs=-1, verbose=0)
    lda.fit(frequencies) # extract topic

    words = vector.get_feature_names()

    for topicID, topic in enumerate(lda.components_):
        topics = [words[i] for i in topic.argsort()[:-MODEL_WORDS - 1:-1]] 
        
    return topics

In [6]:
categories = ["collection", "processing", "dissemination", "risk", "protection"]

article_ids = {}

for c in categories:
    article_ids[c] = {}

for c in categories:
    with open('./final_annotations/'+ c +'_org.csv', 'r') as f:
            reader = csv.reader(f)
            for r in reader:
                article_ids[c][r[0]] = r[1]

In [101]:
CLASSIFICATION = "risk"

In [102]:
# keyterms = [
#     "aggregat", "algorithm", "analy", "computing", "extract", "identif", "linking", "mapping",
#     "mining", "predict", "processed", "processing"
    
# ]

# keyterms = [
    
#     "amendment", "anonym", "confidential", "consent", "encrypt", "framework", "guideline",
#     "legal", "legislat", "policy",
# "protect", "regul", "ruling", "safe", "standard", "vpn"
# ]

In [103]:
import random

def get_set(cnt, ids):
    
    copy = ids
    
    random_ids = []
    
#     if len(keys) == 0:
#         return []
    
    for i in range(cnt):
        choice = random.choice(copy)
        random_ids.append(choice)
        copy.remove(choice)
        
    return sorted(random_ids), copy

In [108]:
# len_test = round(0.1 * len(article_ids[CLASSIFICATION]))
# len_valid = round(0.1 * len(article_ids[CLASSIFICATION]))
# len_train = len(article_ids[CLASSIFICATION]) - len_test - len_valid

ones = [i for i in article_ids[CLASSIFICATION].keys() if int(article_ids[CLASSIFICATION][i]) == 1]
COPY_ONES = ones
zeroes = [i for i in article_ids[CLASSIFICATION].keys() if int(article_ids[CLASSIFICATION][i]) == 0]
#print(len(zeroes))
LEN_ONES = len(ones)
LEN_ZEROES = len(zeroes)

test_data, ones = get_set(round(0.1 * LEN_ONES), ones)# get 1s
#print(len(test_data))
temp, zeroes = get_set(round(0.1 * LEN_ZEROES), zeroes) # get 0s
for i in temp:
    test_data.append(i)
    
validation_data, ones = get_set(round(0.1 * LEN_ONES), ones)# get 1s
#print(len(validation_data))
temp, zeroes = get_set(round(0.1 * LEN_ZEROES), zeroes) # get 0s
for i in temp:
    validation_data.append(i)
    
training_data = ones
for i in zeroes:
    training_data.append(i)

In [109]:
# TEST IF ONLY ABSTRACT
# db methods
def select_article(curs, db, aid):
    
    curs.execute("""SELECT title, content 
                FROM articles 
                WHERE article_id = '""" + aid + """' 
                """)
                #LIMIT 200""") 
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[aid] = {
            "title": r[0],
            "content": r[1],
            "verdict": article_ids[CLASSIFICATION][aid]
        }
    
    return arts[aid]



In [110]:
final_test_data = {}
final_training_data = {}
final_validation_data = {}

for i in test_data:
    temp = select_article(mycursor, mydb, i)
    final_test_data[i] = {
        "id": i,
        "label": int(temp["verdict"]),
        "text": temp["title"] + " " + temp["content"]
    }
    
for i in training_data:
    temp = select_article(mycursor, mydb, i)
    final_training_data[i] = {
        "id": i,
        "label": int(temp["verdict"]),
        "text": temp["title"] + " " + temp["content"]
    }
    
for i in validation_data:
    temp = select_article(mycursor, mydb, i)
    final_validation_data[i] = {
        "id": i,
        "label": int(temp["verdict"]),
        "text": temp["title"] + " " + temp["content"]
    }

In [111]:
df_test = pd.DataFrame.from_dict(final_test_data).T
df_training = pd.DataFrame.from_dict(final_training_data).T
df_validation = pd.DataFrame.from_dict(final_validation_data).T

print('Number of test sentences: {:,}'.format(df_test.shape[0]))
print('Number of validation sentences: {:,}'.format(df_validation.shape[0]))
print('Number of training sentences: {:,}'.format(df_training.shape[0]))

df_training.sample(10)

Number of test sentences: 20
Number of validation sentences: 20
Number of training sentences: 160


,id,label,text
DT_4614,DT_4614,0,Voters are more predictable than they think Ap...
SMH_956,SMH_956,0,Immigration edict leaves child humiliated at g...
SMH_2358,SMH_2358,0,Revenge porn: caught in a web of spite Hundred...
TDP_716,TDP_716,1,Adverts by app - Dotcom looks to future KIM DO...
SMH_2236,SMH_2236,1,Pub scanners raise privacy concerns More than ...
TS_4155,TS_4155,0,Your face could be your next password;Shopping...
TG_13948,TG_13948,0,A historic antitrust hearing in Congress has p...
AFR_1346,AFR_1346,0,"Intrusive privacy breach Your item (""ABS slamm..."
TG_7270,TG_7270,1,Police face new ethical dilemma in increasingl...
TG_8142,TG_8142,1,The injunction is back: entertainer blocks ext...


In [112]:
def get_frequencies(m):
    
    words = []
    
    
    for w in m: # for each word in model
        for k in keyterms: # for each keyterm in a category
            if k in w[:len(k)]: # get only the words that begin with
                words.append({
                     "keyword": k, 
                     "model_word": w
                 })
    return words

In [113]:
keyterms =[ "cloud", "collect", "databas", "harvest", "logging", "monitor", "record", 
"retention", "snoop", "storage", "stored", "surveil",
"track"]

# keyterms =["access","attack","breach","disclos","hack","intercept","intrud","intrus","leak",
# "risk","snoop","spy","spying","stalk","surveil","theft","track"]

In [114]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def get_conf_matrix(x, y):
    return confusion_matrix(np.array(x), y).ravel()

def get_recall(x, y):
    return recall_score(x, y, average='macro')
    
def get_precision(x, y):
    return precision_score(x, y, average='macro')

def get_f1_score(x, y):
    return f1_score(x, y, average='macro')

In [115]:
import numpy as np

In [20]:
SET = final_training_data

In [21]:
true_labels = {}

for a in SET.keys():
    true_labels[a] = SET[a]["label"]

In [25]:
predicted_labels = {}
models = {}
frqs = {}

for a in SET.keys():
    m = model([SET[a]["text"]])
    models[a] = m
    f = get_frequencies(m)
    frqs[a] = f
    isc = is_category(f)
    predicted_labels[a] = isc

In [26]:
ids = sorted(true_labels.keys())

x_data = [true_labels[i] for i in ids]
y_data = [predicted_labels[i] for i in ids]

tn, fp, fn, tp = get_conf_matrix(x_data, y_data)
print(
    "\nTN:", tn,
    "\nTP:", tp,
    "\nFN:", fn,
    "\nFP:", fp    
)

recall = round(get_recall(x_data, y_data), 3)
precision = round(get_precision(x_data, y_data), 3)
f1 = round(get_f1_score(x_data, y_data), 3)

print("Recall:", recall)
print("Precision:", precision)
print("F1-score:", f1)


TN: 148 
TP: 9 
FN: 43 
FP: 0
Recall: 0.587
Precision: 0.887
F1-score: 0.584


In [24]:
def is_category(frq): # based on frequency
    
    if len(frq) >= 3:
        return 1
    else:
        return 0


In [27]:
def model2(cnt):
    frequencies = vector.fit_transform(cnt)

    lda = LDA(n_components=NUMBER_TOPICS2, n_jobs=-1, verbose=0)
    lda.fit(frequencies) # extract topic

    words = vector.get_feature_names()

    topics = []
    
    for topicID, topic in enumerate(lda.components_):
        t = [words[i] for i in topic.argsort()[:-MODEL_WORDS - 1:-1]] 
        for i in t:
            topics.append(i)
        
    return list(set(topics))

In [28]:
predicted_labels = {}
models = {}
frqs = {}

for a in SET.keys():
    m = model2([SET[a]["text"]])
    models[a] = m
    f = get_frequencies(m)
    frqs[a] = f
    isc = is_category(f)
    predicted_labels[a] = isc

In [29]:
ids = sorted(true_labels.keys())

x_data = [true_labels[i] for i in ids]
y_data = [predicted_labels[i] for i in ids]

tn, fp, fn, tp = get_conf_matrix(x_data, y_data)
print(
    "\nTN:", tn,
    "\nTP:", tp,
    "\nFN:", fn,
    "\nFP:", fp    
)

recall = round(get_recall(x_data, y_data), 3)
precision = round(get_precision(x_data, y_data), 3)
f1 = round(get_f1_score(x_data, y_data), 3)

print("Recall:", recall)
print("Precision:", precision)
print("F1-score:", f1)


TN: 147 
TP: 10 
FN: 42 
FP: 1
Recall: 0.593
Precision: 0.843
F1-score: 0.595


In [141]:
# # collect keywords from the training set

# keywords_from = final_training_data

In [142]:
# training_contents = {} # all articles in training data
# ctr = 0
# for a in keywords_from.keys():
#     training_contents[ctr] = keywords_from[a]["text"]
#     ctr += 1

# #temp = vector.fit_transform([keywords_from["AFR_1004"]["text"]])


In [143]:
# training_df = pd.DataFrame.from_dict(training_contents, orient="index", columns=["Article"])

In [144]:
# training_df.sample(5)

In [210]:
rp_ids = []

with open('./rp.csv', 'r') as f:
        reader = csv.reader(f)
        for r in reader:
            rp_ids.append(r[0])

In [214]:
# TEST IF ONLY ABSTRACT
# db methods
def select_cnt(curs, db, aid):
    
    curs.execute("""SELECT title, content 
                FROM articles 
                WHERE article_id = '""" + aid + """' 
                """)
                #LIMIT 200""") 
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[aid] = {
            "content": r[0] + " " + r[1]
        }
    
    return arts[aid]



In [215]:
# ones only
training = {}
ctr = 0
for a in rp_ids:
    training[ctr] = select_cnt(mycursor, mydb, a)["content"]
    ctr += 1

In [216]:
training_df = pd.DataFrame.from_dict(training, orient="index", columns=["Article"])
print(training_df.shape[0])

71


In [217]:
training_df.sample(5)

,Article
26,Web laws are bad for Google - and us FULL TEXT...
11,'PEOPLE IN CRISIS' REPORT;Equip police officer...
69,"Blagojevich starts his prison term today ""I'll..."
53,The injunction is back: entertainer blocks ext...
44,Royals awarded damages over topless photos FRA...


In [218]:
cv = CountVectorizer(max_df = 0.80, min_df = 5, stop_words="english")

In [219]:
cv_fit = cv.fit_transform(training_df.Article)

In [220]:
cv_fit

<71x812 sparse matrix of type '<class 'numpy.int64'>'
	with 7339 stored elements in Compressed Sparse Row format>

In [221]:
#help(LDA)

In [226]:
lda = LDA(n_components = 2, max_iter = 20)

In [227]:
lda.fit(cv_fit)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=20,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [228]:
len(lda.components_)

2

In [229]:
#article_ids[CLASSIFICATION].keys()

In [230]:
feature = cv.get_feature_names()

In [231]:
len(feature)

812

In [234]:
for ind, topic in enumerate(lda.components_):
    
    print("top 50 topics", ind)
    top_50 = topic.argsort()[-200:]
    print(sorted([feature[i] for i in top_50]))
    print()
   

top 50 topics 0
['access', 'according', 'act', 'action', 'actions', 'added', 'administration', 'agency', 'ago', 'air', 'asked', 'attacks', 'australia', 'australian', 'based', 'believe', 'better', 'big', 'book', 'box', 'breach', 'breaches', 'british', 'business', 'called', 'calls', 'canada', 'car', 'case', 'cent', 'changes', 'citizens', 'civil', 'collection', 'comment', 'commission', 'commissioner', 'committee', 'communications', 'companies', 'company', 'complaint', 'complaints', 'concerns', 'conduct', 'consent', 'consumer', 'control', 'countries', 'country', 'data', 'day', 'department', 'details', 'did', 'digital', 'don', 'drivers', 'driving', 'electronic', 'ethics', 'europe', 'european', 'facebook', 'far', 'federal', 'firm', 'foreign', 'free', 'freedom', 'general', 'giant', 'given', 'global', 'going', 'good', 'google', 'government', 'governments', 'gps', 'group', 'help', 'human', 'idea', 'including', 'independent', 'industry', 'information', 'inquiry', 'insurance', 'intelligence', 'in

In [64]:
training_df.T["AFR_2227"]

Article    Local investors stick with Facebook Local fund...
Name: AFR_2227, dtype: object

In [56]:
topic.argmax()

4660

In [35]:


for topicID, topic in enumerate(lda.components_):
    topics = [words[i] for i in topic.argsort()[:-MODEL_WORDS - 1:-1]] 

<1x231 sparse matrix of type '<class 'numpy.int64'>'
	with 231 stored elements in Compressed Sparse Row format>